In [1]:
import gym
import numpy as np
import torch

from stable_baselines3 import SAC, PPO, HerReplayBuffer
from stable_baselines3.common.buffers import DictReplayBuffer
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.monitor import Monitor

from stable_baselines3.common import results_plotter
from stable_baselines3.common.results_plotter import load_results, ts2xy, plot_results
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.callbacks import BaseCallback

import numpy as np
import gym

from Lorenz96_Env_ICs import Lorenz96Env
from stable_baselines3 import PPO
import time 
import matplotlib.pyplot as plt
from typing import Callable

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
def make_env(env_id, rank, integ_step, noise, seed=0):
    """
    Utility function for multiprocessed env.

    :param env_id: (str) the environment ID
    :param num_env: (int) the number of environments you wish to have in subprocesses
    :param seed: (int) the inital seed for RNG
    :param rank: (int) index of the subprocess
    """
    def _init():
        env = Lorenz96Env(integ_steps = integ_step)
        env.seed(seed + rank)
        return env
    set_random_seed(seed+ rank)
    return _init

In [3]:
num_cpu = 70  # Number of processes to use
# Create the vectorized environment
env = SubprocVecEnv([make_env('Lorenz', rank=i, integ_step=int(4), noise=1.0) for i in range(num_cpu)])
env = VecNormalize(env, norm_obs=True, norm_reward=False)

torch.backends.cudnn.benchmark = True

neuron = 128
lr = 0.001
gamma = 0.1
tSteps = 3000000
bs = 100
max_grad_norms = 0.9
vf = 0.8 # 0.7


/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [4]:
# model = PPO.load('trainedModels_L96_assimSteps4_obsNosie1.0_ICs_v2/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps5000000_assimStep4_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/'+'PPO_Lorenz96', env=env)
model = PPO.load('trainedModels_L96_assimSteps4_obsNosie1.0_Markov1Step_TrueNorm/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps3000000_assimStep4_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/'+'PPO_Lorenz96', env=env)
obs = env.reset()

solution = []
ref_soln = []
rewards_ = []

for _ in range(int(5000)):
    # action, _states = model.predict(obs, deterministic=True)
    action, _states = model.predict(obs, deterministic=False)
    obs, rewards, dones, info = env.step(action)
    trueObs = env.get_original_obs()
    trueRew = env.get_original_reward()
    trueRef = env.env_method("_get_ref", 1)

    solution.append(trueObs)
    ref_soln.append(trueRef)
    rewards_.append(trueRew)

sol2 = np.asarray(solution)
solR = np.asarray(ref_soln)


/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
/home/hammoumr/anaconda3/envs/sb3_v2/lib/python3.8/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting 

In [5]:

counter = 0
x = np.zeros((int(5000), 70, 40))
xR = np.zeros((int(5000), 70, 40))
RMSE  = np.zeros((int(5000), 70))
RRMSE = np.zeros((int(5000), 70))

x  = sol2[:, :, -40:]
xR = solR[:, :, -40:]

RMSE  = 0.85*np.sqrt(np.mean((x-xR)**2, axis=-1))
RRMSE = RMSE / np.sqrt(np.mean((xR)**2, axis=-1))

print(RRMSE)

[[0.37223686 0.39042489 0.37999588 ... 0.37641691 0.39567139 0.36099298]
 [0.37913869 0.4959138  0.4858621  ... 0.51446034 0.47657588 0.47899909]
 [0.37442231 0.48569906 0.45553013 ... 0.49077387 0.42216538 0.45900302]
 ...
 [0.63237085 0.49135618 0.58188445 ... 0.56629461 0.51966012 0.67348925]
 [0.61827975 0.50263267 0.60796512 ... 0.59624257 0.49516126 0.67770587]
 [0.59354902 0.50486573 0.55579836 ... 0.77246489 0.49891462 0.62813266]]


In [6]:

# plt.clf()
# plt.figure(figsize=(12, 6))
# fig1 = plt.gcf()

# plt.subplot(1, 3, 2)
# plt.plot(t1, x)  # Plot some data on the axes.
# # plt.xlim((0, 1700))
# plt.subplot(1, 3, 1)
# plt.plot(t1, xR)  # Plot some data on the axes.
# # plt.xlim((0, 1700))
# plt.subplot(1, 3, 3)
# plt.plot(t1, x-xR)  # Plot some data on the axes.
# plt.show()


In [7]:

# plt.clf()
# plt.figure(figsize=(12, 6))
# fig1 = plt.gcf()

# plt.subplot(1, 3, 2)
# plt.plot(t1, y)  # Plot some data on the axes.
# # plt.xlim((0, 1700))
# plt.subplot(1, 3, 1)
# plt.plot(t1, yR)  # Plot some data on the axes.
# # plt.xlim((0, 1700))
# plt.subplot(1, 3, 3)
# plt.plot(t1, y-yR)  # Plot some data on the axes.
# plt.show()


In [8]:
model.save('data/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps'+str(tSteps)+'_assimStep50_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/PPO_Gaussian_lossDifference_L96_fullObs_GaussianNoise_every50')

np.savez('data/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps'+str(tSteps)+'_assimStep50_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/sol2', sol2=sol2)
np.savez('data/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps'+str(tSteps)+'_assimStep50_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/solR', solR=solR)

np.savez('data/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps'+str(tSteps)+'_assimStep50_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/pred', x=x)
np.savez('data/'+'neurons'+str(neuron)+'_LR'+str(lr)+'_Gamma'+str(gamma)+'_BS'+str(bs)+'_totSteps'+str(tSteps)+'_assimStep50_obsNosie1.0_maxgradnorm'+str(max_grad_norms)+'_vfcoeff'+str(vf)+'/ref',  xR=xR)


/data1/RL/test_recallWhatIdid/PerfectModels_testNoise/Full_MDP_allobserved/stable-baselines3/stable_baselines3/common/save_util.py:278: UserWarning: Path 'data/neurons128_LR0.001_Gamma0.1_BS100_totSteps3000000_assimStep50_obsNosie1.0_maxgradnorm0.9_vfcoeff0.8' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
